<a href="https://colab.research.google.com/github/Alfonzo1114/MiniMax/blob/main/Python_Game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Minimax algorithm application for a 2-player board game (human vs computer)

In [ ]:
import pandas as pd
import numpy as np
# G1 Compture, G2 Human
turno = True # giocatore 1: TRUE o giocatore 2: FALSE
# Dimensione della scheda
N = 8
#Creazione della scheda casuale
scheda = pd.DataFrame(np.random.randint(1, 2*N, size = (N, N)))
# Creazione de la matriz logica
occupato = pd.DataFrame(np.zeros(shape = (N, N)))
# Creazione della matrice giocatori per far riempire dei colorei nel corso della giocata
# ci sono dei valori 'G1' o 'G2' o 'vuoto'
giocatori = occupato.copy()

for i in range(N):
    occupato = occupato.astype({i: bool})
    for k in range(N):
        giocatori.iloc[i, k] = 'vuoto'
display(giocatori)

# posizione originale G1
P1 = [0, 0]
# posizione originale G2
P2 = [N - 1, N - 1]
# posizioni iniziali occupate prendono il valore True
occupato.iloc[P1[0], P1[1]] = True
giocatori.iloc[P1[0], P1[1]] = 'G1'
occupato.iloc[P2[0], P2[1]] = True
giocatori.iloc[P2[0], P2[1]] = 'G2'
#Livello di spessore che utilizza l'algoritmo
spessore = 6
punteggio = scheda.iloc[P1[0], P1[1]] - scheda.iloc[P2[0], P2[1]] # + vince G1, - vince G2, 0 pareggio.

,0,1,2,3,4,5,6,7
0,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
1,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
2,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
3,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
4,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
5,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
6,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto
7,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto,vuoto


# *Matrix creation, node and the minimax algorithm as well as the tree creation*

In [ ]:
class Mastice:
    def __init__(self, spazio=100):
        self.info = []
        self.spazio = spazio

    def enqueue(self, n):
        if len(self.info) < self.spazio:
            self.info.insert(0, n)

    def dequeue(self):
        return self.info.pop()

    def peek(self):
        return self.info[-1]

    def empty(self):
        return self.info == []

    def full(self):
        return len(self.info) == self.spazio

    def count(self):
        return len(self.info)

    def print(self):
        print(self.info[::-1])

class Nodo:
    # Raiz
    def __init__(self, P1, P2, occupato, punteggio):
        self.occupato = occupato
        self.P1 = P1
        self.P2 = P2
        self.sin = None
        self.des = None
        self.sop = None
        self.sot = None
        self.punteggio = punteggio

    def albero(self, spessore):
        # turno: True G1, False: G2
        sterno = Mastice()
        sterno.enqueue(self)
        sterno.enqueue('*')
        turno = True
        c = 0
        while not sterno.empty() or spessore >= c:
            corrente = sterno.dequeue()
            if corrente == '*':
                c += 1
                corrente = sterno.dequeue()
                if sterno.count() != 0:
                    sterno.enqueue('*')

            if turno:  # giocatore 1
                x, y = corrente.P1[0], corrente.P1[1]
            else:
                x, y = corrente.P2[0], corrente.P2[1]
            # convalida se sinistra è valida
            if y - 1 < 0 or corrente.occupato.iloc[x, y  - 1]:
                corrente.sin = None
            else:
                nuovo_occupato = corrente.occupato.copy()
                nuovo_occupato.iloc[x, y - 1] = True
                if turno:  # turno G1
                    nuevo_punteggio = corrente.punteggio + scheda.iloc[x, y - 1]
                    corrente.sin = Nodo([x, y - 1], corrente.P2, nuovo_occupato, nuevo_punteggio)
                else:  # turno G2
                    nuevo_punteggio = corrente.punteggio - scheda.iloc[x, y - 1]
                    corrente.sin = Nodo(corrente.P1, [x, y - 1], nuovo_occupato, nuevo_punteggio)
                sterno.enqueue(corrente.sin)  # EnMastice

            # convalida se destra è valida
            if y + 1 == N or occupato.iloc[x, y + 1]:
                corrente.des = None
            else:
                nuovo_occupato = corrente.occupato.copy()
                nuovo_occupato.iloc[x, y + 1] = True
                if turno:  # turno G1
                    nuevo_punteggio = corrente.punteggio + scheda.iloc[x, y + 1]
                    corrente.des = Nodo([x, y + 1], corrente.P2, nuovo_occupato, nuevo_punteggio)
                else:  # turno G2
                    nuevo_punteggio = corrente.punteggio - scheda.iloc[x, y + 1]
                    corrente.des = Nodo(corrente.P1, [x, y + 1], nuovo_occupato, nuevo_punteggio)
                sterno.enqueue(corrente.des)  # EnMastice

            # convalida se sotto è valida
            if x - 1 < 0 or occupato.iloc[x - 1, y]:
                self.sot = None
            else:
                nuovo_occupato = corrente.occupato.copy()
                nuovo_occupato.iloc[x - 1, y] = True
                if turno:  # turno G1
                    nuevo_punteggio = corrente.punteggio + scheda.iloc[x - 1, y]
                    corrente.sot = Nodo([x - 1, y], corrente.P2, nuovo_occupato, nuevo_punteggio)
                else:  # turno G2
                    nuevo_punteggio = corrente.punteggio - scheda.iloc[x - 1, y]
                    corrente.sot = Nodo(corrente.P1, [x - 1, y], nuovo_occupato, nuevo_punteggio)
                sterno.enqueue(corrente.sot)

            # convalida se sopra è valida
            if x + 1 == N or occupato.iloc[x + 1, y]:
                self.sop = None
            else:
                nuovo_occupato = corrente.occupato.copy()
                nuovo_occupato.iloc[x + 1, y] = True
                if turno:  # turno G1
                    nuevo_punteggio = corrente.punteggio + scheda.iloc[x + 1, y]
                    corrente.sop = Nodo([x + 1, y], corrente.P2, nuovo_occupato, nuevo_punteggio)
                else:  # turno G2
                    nuevo_punteggio = corrente.punteggio - scheda.iloc[x + 1, y]
                    corrente.sop = Nodo(corrente.P1, [x + 1, y], nuovo_occupato, nuevo_punteggio)
                sterno.enqueue(corrente.sop)
            turno = not turno  # cambio turno
            return corrente

    def minimax(self, tipo = True, spessore = spessore):
        if self.sin is not None:
            i, _ = self.sin.minimax(not tipo, spessore - 1)
        if self.des is not None:
            d, _ = self.des.minimax(not tipo, spessore - 1)
        if self.sop is not None:
            ar, _ = self.sop.minimax(not tipo, spessore - 1)
        if self.sot is not None:
            ab, _ = self.sot.minimax(not tipo, spessore - 1)

        # foglio
        if self.sin is None:
            if self.des is None:
                if self.sop is None:
                    if self.sot is None:
                        return self, 'senza mossa'

        # Un solo resultado
        if self.sin is not None and self.des is None and self.sot is None and self.sop is None:
            return i, 'sinistra'
        if self.des is not None and self.sin is None and self.sot is None and self.sop is None:
            return d, 'destra'
        if self.sot is not None and self.des is None and self.sin is None and self.sop is None:
            return ab, 'sotto'
        if self.sop is not None and self.des is None and self.sin is None and self.sot is None:
            return ar, 'sopra'

        # Dos o mas resultados posibles

        if self.sin is not None:  # Izquierdo y
            # (SIN, DES)
            if self.des is not None:
                if self.sot is None and self.sop is None:
                    if i.punteggio > d.punteggio:
                        return i, 'sinistra'  # Si sinistra es el maximo
                    return d, 'destra'
                # (SIN,DES,SOT)
                if self.sot is not None and self.sop is None:
                    if i.punteggio > d.punteggio and i.punteggio > ab.punteggio:
                        return i, 'sinistra'
                    if ab.punteggio > d.punteggio:
                        return ab, 'sotto'
                    return d, 'destra'
                # (SIN,DES,ARR)
                if self.sop is not None and self.sot is None:
                    if i.punteggio > d.punteggio and i.punteggio > ar.punteggio:
                        return i, 'sinistra'
                    if ar.punteggio > d.punteggio:
                        return ar, 'sopra'
                    return d, 'destra'
                    # (SIN, SOT)
            if self.sot is not None:
                if self.des is None and self.sop is None:
                    if i.punteggio > ab.punteggio:
                        return i, 'sinistra'  # Si sinistra es el maximo
                    return ab, 'sotto'
                    # Caso (SIN, SOT, ARR)
                if self.sop is not None:
                    if i.punteggio > ab.punteggio and i.punteggio > ar.punteggio:
                        return i, 'sinistra'
                    if ab.punteggio > ar.punteggio:
                        return ab, 'sotto'
                    return ar, 'sopra'
            # (SIN, ARR)
            if self.sop is not None:
                if i.punteggio > ar.punteggio:
                    return i, 'sinistra'  # Si sinistra es el maximo
                return ar, 'sopra'
        # Aca significa que sinistra esta vacio
        if self.des is not None:
            # (DES, SOT)
            if self.sot is not None:
                if self.sop is None:
                    if d.punteggio > ab.punteggio:
                        return d, 'destra'
                    return ab, 'sotto'
                # (DES, SOT, ARR)
                if d.punteggio > ab.punteggio and d.punteggio > ar.punteggio:
                    return d, 'destra'
                if ab.punteggio > ar.punteggio:
                    return ab, 'sotto'
                return ar, 'sopra'
            # (DES, ARR)
            if self.sop is not None:
                if d.punteggio > ar.punteggio:
                    return d, 'destra'
                return ar, 'sopra'

        if self.sop is not None:
            # (ARR, SOT)
            if ar.punteggio > ab.punteggio:
                return ar, 'sopra'
            return ab, 'sotto'


#Main Body

In [ ]:
def svuotare(giocatore, occupato):
    if (giocatore[0] - 1 < 0 or occupato.iloc[giocatore[0] - 1, giocatore[1]]) and \
        (giocatore[0] + 1 == N or occupato.iloc[giocatore[0] + 1, giocatore[1]]) and \
        (giocatore[1] + 1 == N or occupato.iloc[giocatore[0], giocatore[1] + 1]) and \
        (giocatore[1] - 1 < 0 or occupato.iloc[giocatore[0], giocatore[1] - 1]):
        return True
    return False

def validare(posizione):
    mensaje = 'mossa non permessa'
    while True:
        try:
            mov = int(input('mossa [sinistra: 1, destra: 2, sopra: 3, sotto: 4]: '))
            if mov not in [1, 2, 3, 4]:
                print(mensaje)
            elif mov == 1: # sinistra
                nuovapos = [posizione[0], posizione[1] - 1]
                if nuovapos[1] >= 0 and occupato.iloc[nuovapos[0], nuovapos[1]] != True:
                    return nuovapos
                print(mensaje)
            elif mov == 2: # Derecha
                nuovapos = [posizione[0], posizione[1] + 1]
                if nuovapos[1] <= N and occupato.iloc[nuovapos[0], nuovapos[1]] != True:
                    return nuovapos
                print(mensaje)
            elif mov == 3: # sopra
                nuovapos = [posizione[0] - 1, posizione[1]]
                if nuovapos[0] >= 0 and occupato.iloc[nuovapos[0], nuovapos[1]] != True:
                    return nuovapos
                print(mensaje)
            else: # Abajo
                nuovapos = [posizione[0] + 1, posizione[1]]
                if nuovapos[0] <= N and occupato.iloc[nuovapos[0], nuovapos[1]] != True:
                    return nuovapos
                print(mensaje)
        except:
            print(mensaje)

def Vincitore(punteggio, stato = True):
    # True es para mostrar los resultados cuando todavia no ha terminado el juego
    if stato:
        if punteggio < 0:
            return 'Va vincendo G2'
        elif punteggio > 0:
            return 'Va vincendo G1'
        return 'Pareggiando'
    # Cuando ya se acabe el juego, se invova esta funcion con stato = False
    if punteggio < 0:
        return 'vince G2'
    elif punteggio > 0:
        return 'vince G1'
    return 'pareggio'

# Funcion para agregar colore a cada celda de la matriz giocatori
def colore(_, matricebase):
    '''Da colore ad ogni elemento occupato sia per il giocatore 1 come
    per il 2 ase come la scelta del colore e fondo per ognuno'''
    formatocelle = {'G1': f"background-color: GoldenRod; color: black", 'G2': f"background-color: green; color: white",'vuoto':""}
    return matricebase.replace(formatocelle)

In [ ]:
while not svuotare(P1, occupato) or not svuotare(P2, occupato):
    if turno: # turno de G1
        if svuotare(P1, occupato):
            print('Gioco finito')
            break
        # viene creato pria l'albero con l'informazione de P1, P2, occupato e punteggio
        radice = Nodo(P1, P2, occupato, punteggio)
        radice.albero(spessore)
        #ichiede alla compu su mov. ovvero, il minimax
        _ , mov = radice.minimax()
        if mov == 'sinistra':
            P1 = [P1[0], P1[1] - 1]
        elif mov == 'destra': # destra
            P1 = [P1[0], P1[1] + 1]
        elif mov == 'sopra': # sopra
            P1 = [P1[0] + 1, P1[1]]
        else:
            P1 = [P1[0] - 1, P1[1]] # Abajo
        # hacer el movimiento
        punteggio += scheda.iloc[P1[0], P1[1]]
        occupato.iloc[P1[0], P1[1]] = True
        giocatori.iloc[P1[0], P1[1]] = 'G1'
        #Mostrar scheda y mensaje de quien va vincendo:
        display(scheda.style.apply(colore, matricebase=giocatori, axis=None))
        print(f"punteggio: {punteggio}, {Vincitore(punteggio)}")
        # cambiar turno
        turno = not turno

    else: # G2
        if svuotare(P2, occupato):
            print('Gioco finito')
            break
        # Preguntar el movimiento a G2 y validarlo
        P2 = validare(P2)
        # Hacer movimiento
        punteggio -= scheda.iloc[P2[0], P2[1]]
        occupato.iloc[P2[0], P2[1]] = True
        giocatori.iloc[P2[0], P2[1]] = 'G2'
        #Mostrar scheda y mensaje de quien va vincendo:
        display(scheda.style.apply(colore, matricebase=giocatori, axis=None))
        print(f"punteggio: {punteggio}, {Vincitore(punteggio)}")
        #cambiar turno
        turno = not turno

print(f"punteggio: {punteggio}, {Vincitore(punteggio, False)}")

,0,1,2,3,4,5,6,7
0,10,11,5,15,4,14,13,4
1,11,13,1,8,5,7,15,13
2,12,10,12,1,12,3,3,2
3,3,13,2,7,13,13,4,10
4,13,9,10,14,14,9,10,15
5,6,8,8,5,15,4,4,5
6,13,11,4,12,2,13,5,2
7,4,2,5,12,13,14,2,14


punteggio: 7, Va vincendo G1
mossa [sinistra: 1, destra: 2, sopra: 3, sotto: 4]: 3


,0,1,2,3,4,5,6,7
0,10,11,5,15,4,14,13,4
1,11,13,1,8,5,7,15,13
2,12,10,12,1,12,3,3,2
3,3,13,2,7,13,13,4,10
4,13,9,10,14,14,9,10,15
5,6,8,8,5,15,4,4,5
6,13,11,4,12,2,13,5,2
7,4,2,5,12,13,14,2,14


punteggio: 5, Va vincendo G1


,0,1,2,3,4,5,6,7
0,10,11,5,15,4,14,13,4
1,11,13,1,8,5,7,15,13
2,12,10,12,1,12,3,3,2
3,3,13,2,7,13,13,4,10
4,13,9,10,14,14,9,10,15
5,6,8,8,5,15,4,4,5
6,13,11,4,12,2,13,5,2
7,4,2,5,12,13,14,2,14


...

punteggio: 18, Va vincendo G1
mossa [sinistra: 1, destra: 2, sopra: 3, sotto: 4]: 3


,0,1,2,3,4,5,6,7
0,10,11,5,15,4,14,13,4
1,11,13,1,8,5,7,15,13
2,12,10,12,1,12,3,3,2
3,3,13,2,7,13,13,4,10
4,13,9,10,14,14,9,10,15
5,6,8,8,5,15,4,4,5
6,13,11,4,12,2,13,5,2
7,4,2,5,12,13,14,2,14


punteggio: 14, Va vincendo G1


,0,1,2,3,4,5,6,7
0,10,11,5,15,4,14,13,4
1,11,13,1,8,5,7,15,13
2,12,10,12,1,12,3,3,2
3,3,13,2,7,13,13,4,10
4,13,9,10,14,14,9,10,15
5,6,8,8,5,15,4,4,5
6,13,11,4,12,2,13,5,2
7,4,2,5,12,13,14,2,14


punteggio: 18, Va vincendo G1
punteggio: 18, vince G1
